In [1]:
import pandas as pd
from xgboost import XGBClassifier
from flaml import AutoML

In [2]:
%pwd

'C:\\Users\\User\\Downloads\\인공지능 경진대회\\09_cls_pipe-20220607T022001Z-001\\09_cls_pipe\\train\\train'

In [3]:
import os
os.listdir()

['.ipynb_checkpoints',
 'final_train.csv',
 'Random Forest.ipynb',
 'train.csv',
 'train_data_preprocess.ipynb',
 'XGBoost Tuning - Optuna Example.ipynb',
 'XGBoost Tuning.ipynb',
 'XGBoost.ipynb']

# Read Train Data

In [4]:
df = pd.read_csv('C:\\Users\\User\\Downloads\\인공지능 경진대회\\09_cls_pipe-20220607T022001Z-001\\09_cls_pipe\\train\\train\\train.csv')
df.head()

,leaktype,0HZ,10HZ,20HZ,30HZ,40HZ,50HZ,60HZ,70HZ,80HZ,...,5030HZ,5040HZ,5050HZ,5060HZ,5070HZ,5080HZ,5090HZ,5100HZ,5110HZ,5120HZ
0,out,0,2,2,0,2,0,2,2,2,...,2,5,2,2,5,2,2,5,2,5
1,out,0,0,0,3,0,3,0,0,0,...,0,3,3,3,3,3,3,3,3,6
2,out,0,4,4,4,4,5,4,4,5,...,5,5,6,5,6,6,6,5,6,4
3,out,0,6,5,5,6,5,6,6,5,...,6,6,7,7,5,6,5,5,7,7
4,out,0,3,0,0,3,0,0,3,3,...,3,3,3,3,3,3,3,3,3,3


# Read Test Data

In [5]:
# test_df = pd.read_csv('C:\\Users\\User\\Downloads\\인공지능 경진대회\\09_cls_pipe-20220607T022001Z-001\\09_cls_pipe\\test\\test\\test.csv')
# test_df.head()

# Make Balanced Data Set

In [6]:
df.head()

,leaktype,0HZ,10HZ,20HZ,30HZ,40HZ,50HZ,60HZ,70HZ,80HZ,...,5030HZ,5040HZ,5050HZ,5060HZ,5070HZ,5080HZ,5090HZ,5100HZ,5110HZ,5120HZ
0,out,0,2,2,0,2,0,2,2,2,...,2,5,2,2,5,2,2,5,2,5
1,out,0,0,0,3,0,3,0,0,0,...,0,3,3,3,3,3,3,3,3,6
2,out,0,4,4,4,4,5,4,4,5,...,5,5,6,5,6,6,6,5,6,4
3,out,0,6,5,5,6,5,6,6,5,...,6,6,7,7,5,6,5,5,7,7
4,out,0,3,0,0,3,0,0,3,3,...,3,3,3,3,3,3,3,3,3,3


## Normal Class

In [26]:
normal_df = df[df['leaktype'] == 'normal']
normal_df.head()

,leaktype,0HZ,10HZ,20HZ,30HZ,40HZ,50HZ,60HZ,70HZ,80HZ,...,5030HZ,5040HZ,5050HZ,5060HZ,5070HZ,5080HZ,5090HZ,5100HZ,5110HZ,5120HZ
3600,normal,0,3,5,5,4,5,4,4,4,...,5,5,5,5,6,5,6,5,5,5
3601,normal,0,4,3,4,4,4,4,4,4,...,5,5,5,6,6,6,5,5,6,6
3602,normal,0,4,4,4,4,5,9,4,5,...,6,6,6,5,5,5,6,5,6,6
3603,normal,0,4,4,5,5,5,5,4,4,...,6,5,6,6,6,5,6,7,5,6
3604,normal,0,3,3,3,4,3,3,3,3,...,5,3,4,4,3,3,3,4,4,3


In [27]:
train_normal = normal_df.sample(1600, random_state=11)
train_normal.head()

,leaktype,0HZ,10HZ,20HZ,30HZ,40HZ,50HZ,60HZ,70HZ,80HZ,...,5030HZ,5040HZ,5050HZ,5060HZ,5070HZ,5080HZ,5090HZ,5100HZ,5110HZ,5120HZ
14109,normal,0,4,3,4,5,4,4,4,3,...,4,5,5,6,6,5,4,6,5,5
15001,normal,0,4,5,4,4,4,4,4,4,...,4,6,6,6,5,6,5,5,6,6
17794,normal,0,5,4,6,4,6,7,8,7,...,8,8,8,8,9,9,8,9,7,9
15534,normal,0,5,3,4,4,5,4,4,4,...,5,5,6,6,6,6,6,5,6,5
12255,normal,0,5,5,5,5,4,3,5,4,...,7,8,7,6,7,6,8,6,7,7


In [28]:
print(train_normal.shape[0])

1600


## Other Class

In [29]:
other_df = df[df['leaktype'] == 'other']
other_df.head()

,leaktype,0HZ,10HZ,20HZ,30HZ,40HZ,50HZ,60HZ,70HZ,80HZ,...,5030HZ,5040HZ,5050HZ,5060HZ,5070HZ,5080HZ,5090HZ,5100HZ,5110HZ,5120HZ
21600,other,0,3,3,3,3,3,3,3,3,...,5,5,5,3,3,3,3,5,5,5
21601,other,0,7,5,5,7,7,8,12,17,...,5,5,5,5,5,5,3,5,5,5
21602,other,0,3,3,4,4,4,4,3,4,...,6,4,5,6,5,6,6,6,6,5
21603,other,0,3,1,3,3,3,3,3,3,...,4,4,6,4,4,6,4,4,4,4
21604,other,0,11,10,14,14,19,31,81,34,...,4,4,3,3,3,3,3,3,3,4


In [30]:
train_other = other_df.sample(1600, random_state=11)
train_other.head()

,leaktype,0HZ,10HZ,20HZ,30HZ,40HZ,50HZ,60HZ,70HZ,80HZ,...,5030HZ,5040HZ,5050HZ,5060HZ,5070HZ,5080HZ,5090HZ,5100HZ,5110HZ,5120HZ
26034,other,0,0,0,0,3,0,0,3,3,...,3,0,0,3,3,0,3,3,0,3
22071,other,0,2,2,2,0,2,2,2,2,...,2,2,2,2,2,2,2,2,2,2
25092,other,0,6,7,4,6,8,8,13,17,...,4,4,4,4,4,4,4,4,4,4
27458,other,0,10,8,11,11,10,10,10,10,...,36,38,37,33,29,26,25,25,27,22
25116,other,0,3,3,5,5,3,3,3,3,...,5,6,5,5,5,6,6,3,5,6


In [31]:
print(train_other.shape[0])

1600


## Noise Class

In [32]:
noise_df = df[df['leaktype'] == 'noise']
noise_df.head()

,leaktype,0HZ,10HZ,20HZ,30HZ,40HZ,50HZ,60HZ,70HZ,80HZ,...,5030HZ,5040HZ,5050HZ,5060HZ,5070HZ,5080HZ,5090HZ,5100HZ,5110HZ,5120HZ
28600,noise,0,2,2,2,2,2,2,2,2,...,5,5,5,5,2,2,5,2,2,2
28601,noise,0,0,0,0,0,7,3,3,3,...,0,0,0,0,0,0,0,0,0,0
28602,noise,0,3,3,3,3,3,3,3,3,...,5,5,5,5,5,5,5,5,5,5
28603,noise,0,4,4,4,8,13,21,25,34,...,6,6,4,4,6,4,6,6,6,6
28604,noise,0,4,4,4,4,4,4,7,7,...,0,0,0,0,0,0,0,0,0,0


In [33]:
train_noise = noise_df.sample(1600, random_state=11)
train_noise.head()

,leaktype,0HZ,10HZ,20HZ,30HZ,40HZ,50HZ,60HZ,70HZ,80HZ,...,5030HZ,5040HZ,5050HZ,5060HZ,5070HZ,5080HZ,5090HZ,5100HZ,5110HZ,5120HZ
31208,noise,0,6,7,9,11,11,17,15,17,...,2,4,2,4,4,4,4,4,4,4
29187,noise,0,2,2,2,4,2,4,7,9,...,4,4,4,4,4,4,4,4,4,4
31007,noise,0,4,4,4,5,5,5,5,5,...,5,6,7,7,6,6,6,5,5,5
30287,noise,0,2,2,2,2,2,2,2,2,...,7,7,7,7,5,5,5,5,5,2
31515,noise,0,5,5,5,5,11,16,13,13,...,8,11,8,8,8,8,5,8,8,11


In [34]:
print(train_noise.shape[0])

1600


## Out Class

In [35]:
out_df = df[df['leaktype'] == 'out']
out_df.head()

,leaktype,0HZ,10HZ,20HZ,30HZ,40HZ,50HZ,60HZ,70HZ,80HZ,...,5030HZ,5040HZ,5050HZ,5060HZ,5070HZ,5080HZ,5090HZ,5100HZ,5110HZ,5120HZ
0,out,0,2,2,0,2,0,2,2,2,...,2,5,2,2,5,2,2,5,2,5
1,out,0,0,0,3,0,3,0,0,0,...,0,3,3,3,3,3,3,3,3,6
2,out,0,4,4,4,4,5,4,4,5,...,5,5,6,5,6,6,6,5,6,4
3,out,0,6,5,5,6,5,6,6,5,...,6,6,7,7,5,6,5,5,7,7
4,out,0,3,0,0,3,0,0,3,3,...,3,3,3,3,3,3,3,3,3,3


In [36]:
train_out = out_df.sample(1600, random_state=11)
train_out.head()

,leaktype,0HZ,10HZ,20HZ,30HZ,40HZ,50HZ,60HZ,70HZ,80HZ,...,5030HZ,5040HZ,5050HZ,5060HZ,5070HZ,5080HZ,5090HZ,5100HZ,5110HZ,5120HZ
1735,out,0,6,6,6,6,8,8,6,8,...,8,8,8,8,10,10,8,8,8,8
369,out,0,11,13,13,17,16,17,16,19,...,18,18,17,18,17,18,18,18,19,18
1731,out,0,25,26,29,25,30,25,24,29,...,40,49,44,41,42,41,42,38,32,36
175,out,0,0,0,0,0,0,0,0,0,...,2,0,2,0,2,0,0,2,0,0
1164,out,0,2,2,2,3,2,2,3,3,...,3,3,2,2,2,3,3,2,3,3


In [37]:
print(train_out.shape[0])

1600


## In Class

In [38]:
in_df = df[df['leaktype'] == 'in']
in_df.head()

,leaktype,0HZ,10HZ,20HZ,30HZ,40HZ,50HZ,60HZ,70HZ,80HZ,...,5030HZ,5040HZ,5050HZ,5060HZ,5070HZ,5080HZ,5090HZ,5100HZ,5110HZ,5120HZ
1800,in,0,18,15,12,12,9,9,9,6,...,0,3,3,3,3,3,3,3,0,3
1801,in,0,3,3,4,4,4,4,4,4,...,7,9,6,9,7,7,6,6,7,6
1802,in,0,0,0,2,2,0,2,2,2,...,2,2,2,2,2,2,2,2,2,2
1803,in,0,17,17,19,17,17,14,19,19,...,39,41,41,50,52,52,47,39,33,36
1804,in,0,4,4,4,4,2,4,4,4,...,4,5,5,4,5,5,5,5,4,5


In [39]:
train_in = in_df.sample(1600, random_state=11)
train_in.head()

,leaktype,0HZ,10HZ,20HZ,30HZ,40HZ,50HZ,60HZ,70HZ,80HZ,...,5030HZ,5040HZ,5050HZ,5060HZ,5070HZ,5080HZ,5090HZ,5100HZ,5110HZ,5120HZ
3535,in,0,6,6,6,6,6,6,7,6,...,7,7,8,8,9,10,8,9,8,9
2169,in,0,6,6,7,6,9,7,9,10,...,20,20,22,20,23,26,31,26,31,33
3531,in,0,3,3,3,3,4,4,6,6,...,3,4,4,6,4,6,4,4,4,6
1975,in,0,3,4,4,4,4,4,4,3,...,5,5,5,5,5,4,5,5,4,5
2964,in,0,4,4,2,4,4,4,4,4,...,5,5,5,7,5,6,5,5,5,6


In [40]:
print(train_in.shape[0])

1600


## Merge Balanced Table

In [41]:
merged_df = pd.concat([train_normal, train_other, train_noise, train_out, train_in], ignore_index=True)
merged_df.head()

,leaktype,0HZ,10HZ,20HZ,30HZ,40HZ,50HZ,60HZ,70HZ,80HZ,...,5030HZ,5040HZ,5050HZ,5060HZ,5070HZ,5080HZ,5090HZ,5100HZ,5110HZ,5120HZ
0,normal,0,4,3,4,5,4,4,4,3,...,4,5,5,6,6,5,4,6,5,5
1,normal,0,4,5,4,4,4,4,4,4,...,4,6,6,6,5,6,5,5,6,6
2,normal,0,5,4,6,4,6,7,8,7,...,8,8,8,8,9,9,8,9,7,9
3,normal,0,5,3,4,4,5,4,4,4,...,5,5,6,6,6,6,6,5,6,5
4,normal,0,5,5,5,5,4,3,5,4,...,7,8,7,6,7,6,8,6,7,7


In [42]:
merged_df.shape

(8000, 514)

In [43]:
merged_df.to_csv('Balanced_Label_Data.csv', index = False)

# Split X_train and y_train

In [24]:
train_data = merged_df.iloc[:,1:]
train_data.head()

,0HZ,10HZ,20HZ,30HZ,40HZ,50HZ,60HZ,70HZ,80HZ,90HZ,...,5030HZ,5040HZ,5050HZ,5060HZ,5070HZ,5080HZ,5090HZ,5100HZ,5110HZ,5120HZ
0,0,5,4,4,4,4,4,4,4,4,...,7,5,6,6,6,6,5,6,8,7
1,0,4,4,5,3,4,4,3,3,3,...,6,6,6,6,6,5,6,6,6,6
2,0,3,4,4,4,4,4,4,3,4,...,6,6,6,6,6,6,7,6,6,7
3,0,5,5,6,6,5,5,4,4,5,...,8,7,6,7,7,7,7,7,8,7
4,0,4,4,5,5,4,4,5,4,5,...,5,5,6,6,6,6,6,6,6,5


In [25]:
# train_x = train_data.iloc[:,:-1]
# train_x.head()

In [26]:
train_y = merged_df.iloc[:,0]
train_y.head()

0    normal
1    normal
2    normal
3    normal
4    normal
Name: leaktype, dtype: object

In [27]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(train_data, train_y, test_size=0.3, random_state=123)

# Tuning Model

## AutoML

In [10]:
# automl = AutoML()

In [44]:
# settings = {
#     "time_budget": -1,  # total running time in seconds
#     "metric": 'macro_f1',  # primary metrics for regression can be chosen from: ['mae','mse','r2']
#     "estimator_list": ['xgboost'],  # list of ML learners; we tune XGBoost in this example
#     "task": 'classification'  # task type 
# }

In [45]:
# automl.fit(X_train=X_train, y_train=y_train, **settings)

[flaml.automl: 06-09 11:56:57] {2376} INFO - task = classification
[flaml.automl: 06-09 11:56:57] {2378} INFO - Data split method: stratified
[flaml.automl: 06-09 11:56:57] {2382} INFO - Evaluation method: holdout
[flaml.automl: 06-09 11:56:57] {2451} INFO - Minimizing error metric: 1-macro_f1
[flaml.automl: 06-09 11:56:57] {2499} WARNING - No search budget is provided via time_budget or max_iter. Training only one model per estimator. To tune hyperparameters for each estimator, please provide budget either via time_budget or max_iter.
[flaml.automl: 06-09 11:56:57] {2591} INFO - List of ML learners in AutoML Run: ['xgboost']
[flaml.automl: 06-09 11:58:44] {3320} INFO - retrain xgboost for 106.9s
[flaml.automl: 06-09 11:58:44] {3327} INFO - retrained model: XGBClassifier(colsample_bylevel=0.7551225121854014,
              colsample_bytree=0.7860755604500558, grow_policy='lossguide',
              learning_rate=0.01872379806270421, max_depth=0, max_leaves=1312,
              min_child_w

In [46]:
# print('Best hyperparmeter config:', automl.best_config)

Best hyperparmeter config: {'n_estimators': 319, 'max_leaves': 1312, 'min_child_weight': 0.001, 'learning_rate': 0.01872379806270421, 'subsample': 0.6890079660561895, 'colsample_bylevel': 0.7551225121854014, 'colsample_bytree': 0.7860755604500558, 'reg_alpha': 0.17028752704343114, 'reg_lambda': 1.4375743264564231}


In [47]:
# print(automl.model.estimator)

XGBClassifier(colsample_bylevel=0.7551225121854014,
              colsample_bytree=0.7860755604500558, grow_policy='lossguide',
              learning_rate=0.01872379806270421, max_depth=0, max_leaves=1312,
              min_child_weight=0.001, n_estimators=12, n_jobs=-1,
              objective='multi:softprob', reg_alpha=0.17028752704343114,
              reg_lambda=1.4375743264564231, subsample=0.6890079660561895,
              tree_method='hist', use_label_encoder=False, verbosity=0)


## GridSearch CV

In [48]:
# from sklearn.model_selection import GridSearchCV

In [49]:
# xg_parameters ={'n_estimators': [400, 500],
#                 'learning_rate':[0.15, 0.2, 0.3], 
#                 'gamma': [4, 5], 
#                 'max_depth' : [3, 4],
#                 'scale_pos_weight' : [3]}

In [50]:
# xgb = XGBClassifier()
# grid_search_xgb = GridSearchCV (estimator = xgb, param_grid = xg_parameters, cv = 5, n_jobs = -1 )

In [51]:
# grid_search_xgb.fit(X_train, y_train)

In [ ]:
# grid_search_xgb.best_params_

# Fit Model

In [52]:
# xgb = XGBClassifier(n_estimators = 319, 
#                     max_leaves = 1312, 
#                     min_child_weight = 0.001, 
#                     learning_rate = 0.01872379806270421, 
#                     subsample = 0.6890079660561895, 
#                     colsample_bylevel = 0.7551225121854014, 
#                     colsample_bytree = 0.7860755604500558, 
#                     reg_alpha = 0.17028752704343114, 
#                     reg_lambda = 1.4375743264564231)

In [28]:
xgb = XGBClassifier(n_estimators=1000)

In [29]:
y_train.value_counts()

noise     1148
in        1128
other     1119
out       1101
normal     824
Name: leaktype, dtype: int64

In [30]:
xgb.fit(X_train, y_train)

XGBClassifier(n_estimators=1000, objective='multi:softprob')

# Prediction

In [31]:
y_pred = xgb.predict(X_test)

# Accuracy Evaluation

In [32]:
from sklearn.metrics import accuracy_score

In [33]:
accuracy = accuracy_score(y_test, y_pred)
print(f'Mean accuracy score: {accuracy:.3}')

Mean accuracy score: 0.731


# F1 Score Evaluation

In [34]:
from sklearn.metrics import f1_score

f1 = f1_score(y_test, y_pred, average = 'macro')
print('f1 score :', f1)

f1 score : 0.7362624028564113


In [35]:
from sklearn.metrics import classification_report

In [36]:
list(df['leaktype'].unique())

['out', 'in', 'normal', 'other', 'noise']

In [37]:
print(classification_report(y_test, y_pred, target_names=list(df['leaktype'].unique())))

              precision    recall  f1-score   support

         out       0.67      0.65      0.66       472
          in       0.73      0.81      0.77       452
      normal       0.85      0.90      0.88       376
       other       0.73      0.71      0.72       481
       noise       0.69      0.63      0.66       499

    accuracy                           0.73      2280
   macro avg       0.73      0.74      0.74      2280
weighted avg       0.73      0.73      0.73      2280



In [38]:
from sklearn.naive_bayes import GaussianNB

In [ ]:
clf = GaussianNB()